In [1]:
import pandas as pd # DataFrame Manipulation Package
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer # Convert a collection of raw documents to a matrix of TF-IDF features
from sklearn.feature_extraction.text import CountVectorizer # Convert a collection of text documents to a matrix of token counts
from sklearn.decomposition import LatentDirichletAllocation # Latent Dirichlet Allocation is a topic model that is used for discovering abstract topics from a collection of documents (variational Bayes algorithm)

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB # The multinomial Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification)

import string # Collection of string operations
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer #Lemmatize using WordNet's built-in morphy function. Returns the input word unchanged if it cannot be found in WordNet.
from nltk import word_tokenize

from nltk.sentiment.util import mark_negation


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [4]:
def to_list(x):
    list_words = x.split(' ')
    return list_words

def to_string(x):
    string = " ".join(x)
    return string

#===============================================================

def punctuation(x):
    for punctuation in string.punctuation:
        x =  x.replace(punctuation, '')
    return x.lower()

def remove_numbers (x):
    words_only = ''.join([i for i in x if not i.isdigit()])
    return words_only

def m_negation(x):
    tokenized = word_tokenize(x)
    without_stopwords = [word for word in tokenized if not word in stop_words]
    tokenized_neg = mark_negation(without_stopwords)
    return tokenized_neg

def remove_stopwords(x):
    tokenized = word_tokenize(x)
    without_stopwords = [word for word in tokenized if not word in stop_words]
    return without_stopwords

def remove_our_stopwords(x):
    tokenized = word_tokenize(x)
    without_stopwords = [word for word in tokenized if not word in OUR_STOPWORDS]
    return without_stopwords

def lemmatize_review(x):
    lemma = WordNetLemmatizer()
    lista = []
    for w in x:
       lista.append(lemma.lemmatize(w))
    return lista

#===============================================================

def count_words(x):
    wordfreq = []
    for w in x:
        wordfreq.append(x.count(w))
    return dict(zip(x, wordfreq))

def total_count(x):
    total_count = {}
    for row in x:
        for key in row.keys():
          if key in total_count:
              total_count[key] += 1
          else:
              total_count[key] = 1
    return pd.DataFrame(sorted(total_count.items(), key=lambda x: x[1], reverse=True)).head(30).T

def round_two(x):
    return str(int(round(x, 2) * 100)) + "%"

def one_or_zero(x):
    # Makes the prediction a binary outpur
    if x > 0.5:
        x = 1
    else:
        x = 0
    return x

#===============================================================

def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], round(topic[i], 2))
                        for i in topic.argsort()[:-10 - 1:-1]])

In [15]:
side_effects = pd.read_csv('/content/drive/MyDrive/Data/frequent_adr.csv')

side_effects.head(3)

,UMLS_ID,MedDRA_Term,MedDRA_ID,Side_Effect
0,C0000727,LT,10000647,Acute abdomen
1,C0000727,PT,10000647,Acute abdomen
2,C0000727,LT,10042784,Syndrome abdominal acute


In [16]:
stop_words = set(stopwords.words('english')) 

side_effects = pd.read_csv('/content/drive/MyDrive/Data/frequent_adr.csv')

pd.DataFrame(side_effects["Side_Effect"])

side_effects["Side_Effect"] = side_effects.Side_Effect.apply(punctuation)
side_effects["Side_Effect"] = side_effects.Side_Effect.apply(remove_numbers)
side_effects["Side_Effect"] = side_effects.Side_Effect.apply(to_list)
side_effects["Side_Effect"] = side_effects.Side_Effect.apply(lemmatize_review)
side_effects["Side_Effect"] = side_effects.Side_Effect.apply(to_string)
side_effects["Side_Effect"] = side_effects.Side_Effect.apply(remove_stopwords)
side_effects = side_effects["Side_Effect"].tolist()

SE_LIST = [] 

for sublist in side_effects: 
    for item in sublist:
        if item not in SE_LIST:
            SE_LIST.append(item)

# SE_LIST

In [17]:
OUR_STOPWORDS = set(stopwords.words('english'))

for effect in SE_LIST: 
    OUR_STOPWORDS.add(effect)

for word in stop_words:
    OUR_STOPWORDS.remove(word)

In [18]:
OUR_STOPWORDS = ['abdominal', 'constipation', 'diarrhea', 'skin', 'rash', 'vertigo', 'dizziness', 'drowsiness',
              'headache', 'mood disorders', 'insomnia', 'mood swings', "no side effect", "good response", "improvement"]

In [19]:
list_merge_SE = ['juan', 'jack', 'peter', 'hendrike', 'headache']

def side_effects_lst(x):
    lista = []
    for i in x:
        y = remove_our_stopwords(i)
        if i not in y:
            lista.append(i)
    return lista

side_effects_lst(list_merge_SE)

['headache']

In [20]:
data = pd.read_csv('/content/drive/MyDrive/Data/drugsComTrain_raw.csv')

data.head(3)

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17


In [11]:
#data = pd.read_csv('drugsComTrain_raw.csv')



data["clean_review"] = data["review"].apply(punctuation)
data['clean_review'] = data.clean_review.apply(remove_numbers)
data['clean_review_lst'] = data.clean_review.apply(to_list)

data["NonStopwords_review_lst"] = data.clean_review.apply(remove_stopwords)
data["NonStopwords_review_str"] = data.NonStopwords_review_lst.apply(to_string)

data["NonStopwords_review_lst_MN"] = data.clean_review.apply(m_negation)
data["NonStopwords_review_str_MN"] = data.NonStopwords_review_lst_MN.apply(to_string)

data["Lemmatized_review_lst"] = data.NonStopwords_review_lst_MN.apply(lemmatize_review)
data["Lemmatized_review_str"] = data.Lemmatized_review_lst.apply(to_string)

data["Lemmatized_review_list"] = data.NonStopwords_review_lst.apply(lemmatize_review)
data["Lemmatized_review"] = data.Lemmatized_review_list.apply(to_string)
data["Lemmatized_review_list"] = data.Lemmatized_review.apply(remove_stopwords)
data["Lemmatized_review"] = data.Lemmatized_review_list.apply(to_string)
# data["our"] = data.Lemmatized_review.apply(remove_our_stopwords)
# data["our_str"] = data.our.apply(to_string)

data["words_count"] = data.Lemmatized_review_list.apply(count_words)

data = data.drop(["clean_review", "clean_review_lst", "NonStopwords_review_lst", "date", "NonStopwords_review_str"], axis = 1)

data["Side_Effects_mention"] = data.Lemmatized_review_list.apply(side_effects_lst)

data.head(3)

,uniqueID,drugName,condition,review,rating,usefulCount,NonStopwords_review_lst_MN,NonStopwords_review_str_MN,Lemmatized_review_lst,Lemmatized_review_str,Lemmatized_review_list,Lemmatized_review,words_count,Side_Effects_mention
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,27,"[side, effect, take, combination, bystolic, mg...",side effect take combination bystolic mg fish oil,"[side, effect, take, combination, bystolic, mg...",side effect take combination bystolic mg fish oil,"[side, effect, take, combination, bystolic, mg...",side effect take combination bystolic mg fish oil,"{'side': 1, 'effect': 1, 'take': 1, 'combinati...",[]
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,192,"[son, halfway, fourth, week, intuniv, became, ...",son halfway fourth week intuniv became concern...,"[son, halfway, fourth, week, intuniv, became, ...",son halfway fourth week intuniv became concern...,"[son, halfway, fourth, week, intuniv, became, ...",son halfway fourth week intuniv became concern...,"{'son': 1, 'halfway': 1, 'fourth': 1, 'week': ...",[good]
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,17,"[used, take, another, oral, contraceptive, pil...",used take another oral contraceptive pill cycl...,"[used, take, another, oral, contraceptive, pil...",used take another oral contraceptive pill cycl...,"[used, take, another, oral, contraceptive, pil...",used take another oral contraceptive pill cycl...,"{'used': 1, 'take': 1, 'another': 1, 'oral': 1...",[]
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,10,"[first, time, using, form, birth, control, im,...",first time using form birth control im glad we...,"[first, time, using, form, birth, control, im,...",first time using form birth control im glad we...,"[first, time, using, form, birth, control, im,...",first time using form birth control im glad we...,"{'first': 3, 'time': 1, 'using': 2, 'form': 1,...",[]
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,37,"[suboxone, completely, turned, life, around, f...",suboxone completely turned life around feel he...,"[suboxone, completely, turned, life, around, f...",suboxone completely turned life around feel he...,"[suboxone, completely, turned, life, around, f...",suboxone completely turned life around feel he...,"{'suboxone': 3, 'completely': 1, 'turned': 1, ...","[good, constipation]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161292,191035,Campral,Alcohol Dependence,"""I wrote my first report in Mid-October of 201...",10,125,"[wrote, first, report, midoctober, alcohol, si...",wrote first report midoctober alcohol since po...,"[wrote, first, report, midoctober, alcohol, si...",wrote first report midoctober alcohol since po...,"[wrote, first, report, midoctober, alcohol, si...",wrote first report midoctober alcohol since po...,"{'wrote': 1, 'first': 2, 'report': 1, 'midocto...",[]
161293,127085,Metoclopramide,Nausea/Vomiting,"""I was given this in IV before surgey. I immed...",1,34,"[given, iv, surgey, immediately, became, anxio...",given iv surgey immediately became anxious cou...,"[given, iv, surgey, immediately, became, anxio...",given iv surgey immediately became anxious cou...,"[given, iv, surgey, immediately, became, anxio...",given iv surgey immediately became anxious cou...,"{'given': 1, 'iv': 1, 'surgey': 1, 'immediatel...",[]
161294,187382,Orencia,Rheumatoid Arthritis,"""Limited improvement after 4 months, developed...",2,35,"[limited, improvement, months, developed, bad,...",limited improvement months developed bad rash ...,"[limited, improvement, month, developed, bad, ...",limited improvement month developed bad rash m...,"[limited, improvement, month, developed, bad, ...",limited improvement month developed bad rash m...,"{'limited': 1, 'improvement': 1, 'month': 1, '...","[improvement, rash]"


In [14]:
data.head(100)

,uniqueID,drugName,condition,review,rating,usefulCount,NonStopwords_review_lst_MN,NonStopwords_review_str_MN,Lemmatized_review_lst,Lemmatized_review_str,Lemmatized_review_list,Lemmatized_review,words_count,Side_Effects_mention
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,27,"[side, effect, take, combination, bystolic, mg...",side effect take combination bystolic mg fish oil,"[side, effect, take, combination, bystolic, mg...",side effect take combination bystolic mg fish oil,"[side, effect, take, combination, bystolic, mg...",side effect take combination bystolic mg fish oil,"{'side': 1, 'effect': 1, 'take': 1, 'combinati...",[]
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,192,"[son, halfway, fourth, week, intuniv, became, ...",son halfway fourth week intuniv became concern...,"[son, halfway, fourth, week, intuniv, became, ...",son halfway fourth week intuniv became concern...,"[son, halfway, fourth, week, intuniv, became, ...",son halfway fourth week intuniv became concern...,"{'son': 1, 'halfway': 1, 'fourth': 1, 'week': ...",[good]
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,17,"[used, take, another, oral, contraceptive, pil...",used take another oral contraceptive pill cycl...,"[used, take, another, oral, contraceptive, pil...",used take another oral contraceptive pill cycl...,"[used, take, another, oral, contraceptive, pil...",used take another oral contraceptive pill cycl...,"{'used': 1, 'take': 1, 'another': 1, 'oral': 1...",[]
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,10,"[first, time, using, form, birth, control, im,...",first time using form birth control im glad we...,"[first, time, using, form, birth, control, im,...",first time using form birth control im glad we...,"[first, time, using, form, birth, control, im,...",first time using form birth control im glad we...,"{'first': 3, 'time': 1, 'using': 2, 'form': 1,...",[]
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,37,"[suboxone, completely, turned, life, around, f...",suboxone completely turned life around feel he...,"[suboxone, completely, turned, life, around, f...",suboxone completely turned life around feel he...,"[suboxone, completely, turned, life, around, f...",suboxone completely turned life around feel he...,"{'suboxone': 3, 'completely': 1, 'turned': 1, ...","[good, constipation]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,45237,Fluoxetine,Major Depressive Disorde,"""I started Prozac as one of my first anti depr...",2,18,"[started, prozac, one, first, anti, depressant...",started prozac one first anti depressants mdd ...,"[started, prozac, one, first, anti, depressant...",started prozac one first anti depressant mdd m...,"[started, prozac, one, first, anti, depressant...",started prozac one first anti depressant mdd m...,"{'started': 2, 'prozac': 1, 'one': 2, 'first':...",[]
96,102810,Aripiprazole,Depression,"""Intake Effexor XR 375 mg, and lorazepam for d...",4,33,"[intake, effexor, xr, mg, lorazepam, depressio...",intake effexor xr mg lorazepam depression anxi...,"[intake, effexor, xr, mg, lorazepam, depressio...",intake effexor xr mg lorazepam depression anxi...,"[intake, effexor, xr, mg, lorazepam, depressio...",intake effexor xr mg lorazepam depression anxi...,"{'intake': 1, 'effexor': 1, 'xr': 1, 'mg': 1, ...",[]
97,60280,NuvaRing,Birth Control,"""I am torn by the Nuvaring. The convenience is...",5,0,"[torn, nuvaring, convenience, great, daily, ti...",torn nuvaring convenience great daily time tak...,"[torn, nuvaring, convenience, great, daily, ti...",torn nuvaring convenience great daily time tak...,"[torn, nuvaring, convenience, great, daily, ti...",torn nuvaring convenience great daily time tak...,"{'torn': 1, 'nuvaring': 3, 'convenience': 1, '...",[]
98,10677,Spironolactone,Acne,"""I&#039;m